# **EE992 Lab 1**
# Intro to Deep Learning Lab

In this lab we are going to basically fork an exisiting DL basics introduction and find out the issues with doing so. Enabling you to troubleshoot these issue later in the class yourself.
Along with that by the end of this lab you should be able to build a NN to solve classification tasks.


In [ ]:
# Only required if you want to save you outputs easily.
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


Test to see if the instance is up and running correctly, checking both the CPU instance and GPU instance.

In [ ]:
!cat /proc/cpuinfo # gives details of CPU

!nvidia-smi # gives details of GPU


## Part 0: Prerequisites:

We recommend that you run this this notebook in the cloud on Google Colab (see link with icon at the top) if you're not already doing so. It's the simplest way to get started. You can also [install TensorFlow locally](https://www.tensorflow.org/install/). But, again, simple is best (with caveats):

[tf.keras](https://www.tensorflow.org/guide/keras) is the simplest way to build and train neural network models in TensorFlow. So, that's what we'll stick with in this tutorial, unless the models neccessitate a lower-level API.

Note that there's [tf.keras](https://www.tensorflow.org/guide/keras) (comes with TensorFlow) and there's [Keras](https://keras.io/) (standalone). You should be using [tf.keras](https://www.tensorflow.org/guide/keras) because (1) it comes with TensorFlow so you don't need to install anything extra and (2) it comes with powerful TensorFlow-specific features.

In [ ]:
# !pip uninstall tensorflow  #just incase you want to change version
# !pip install tensorflow==2.X.0
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten, Dense

# Commonly used modules
import numpy as np
import os
import sys

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import IPython


print(tf.__version__)

## Part 1: Generic Simple ANN:
### Boston Housing Price Prediction with Feed Forward Neural Networks

Let's start with using a fully-connected neural network to do predict housing prices. The following image highlights the difference between regression and classification (see part 2). Given an observation as input, **regression** outputs a continuous value (e.g., exact temperature) and classificaiton outputs a class/category that the observation belongs to.

<img src="https://i.imgur.com/vvSoAzg.jpg" alt="classification_regression" width="400"/>

For the Boston housing dataset, we get 506 rows of data, with 13 features in each. Our task is to build a regression model that takes these 13 features as input and output a single value prediction of the "median value of owner-occupied homes (in $1000)."

Now, we load the dataset. Loading the dataset returns four NumPy arrays:

* The `train_features` and `train_labels` arrays are the *training set*—the data the model uses to learn.
* The model is tested against the *test set*, the `test_features`, and `test_labels` arrays.

In [ ]:
# Load the Boston Housing dataset from the Keras library and split into training and testing sets
(train_features, train_labels), (test_features, test_labels) = keras.datasets.boston_housing.load_data()

# Get per-feature statistics (mean, standard deviation) from the training set to normalize by
train_mean = np.mean(train_features, axis=0)
train_std = np.std(train_features, axis=0)

# Normalize the training features by subtracting the mean and dividing by the standard deviation
train_features = (train_features - train_mean) / train_std

57026/57026 [==============================] - 0s 0us/step


### Build the model

Building the neural network requires configuring the layers of the model, then compiling the model. First we stack a few layers together using `keras.Sequential`. Next we configure the loss function, optimizer, and metrics to monitor. These are added during the model's compile step:

* *Loss function* - measures how accurate the model is during training, we want to minimize this with the optimizer.
* *Optimizer* - how the model is updated based on the data it sees and its loss function.
* *Metrics* - used to monitor the training and testing steps.

Let's build a network with 1 hidden layer of 20 neurons, and use mean squared error (MSE) as the loss function (most common one for regression problems):

In [ ]:

def build_model():
    # Define a simple neural network model
    model = keras.Sequential([
        # First dense layer with 20 neurons and a ReLU activation function
        Dense(20, activation=tf.nn.relu, input_shape=[len(train_features[0])]),
        # Second dense layer with 1 neuron
        Dense(1)
    ])

    # Compile the model with Adam optimizer, mean squared error loss,
    # and metrics for mean absolute error and mean squared error
    model.compile(optimizer=tf.train.AdamOptimizer(),
                  loss='mse',
                  metrics=['mae', 'mse'])
    return model

### Train the model

Training the neural network model requires the following steps:

1. Feed the training data to the model—in this example, the `train_features` and `train_labels` arrays.
2. The model learns to associate features and labels.
3. We ask the model to make predictions about a test set—in this example, the `test_features` array. We verify that the predictions match the labels from the `test_labels` array.

To start training,  call the `model.fit` method—the model is "fit" to the training data:

*In this code, I have defined a callback class called 'PrintDot' that helps make the output of the training less verbose but still shows progress by printing a dot after every epoch. The model is built by calling the 'build_model' function, then an early stopping callback is defined to stop training if the validation loss does not improve for 50 epochs. The model is then trained using the train_features and train_labels, for 100 epochs, with a verbosity level of 1, and a validation split of 10%. The callbacks for early stopping and printing progress are passed to the fit function. The training history is then stored in a pandas DataFrame, and the final root mean square error is calculated on the validation set and printed.*

In [ ]:
# Define a callback class to print the progress of the training
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0:
            print('') # Print a new line after every 100 epochs
        print('.', end='') # Print a dot to indicate progress

# Build the model
model = build_model()

# Define early stopping callback to stop training if the validation loss does not improve for 50 epochs
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

# Train the model with the training data, for 100 epochs, with verbosity level 0, validation split of 10%, and callbacks for early stopping and printing progress
history = model.fit(train_features, train_labels, epochs=100, verbose=0, validation_split = 0.1,
                    callbacks=[early_stop, PrintDot()])

# Create a DataFrame from the history of the training
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

# Calculate the final root mean square error on the validation set
rmse_final = np.sqrt(float(hist['val_mean_squared_error'].tail(1)))

# Print the final root mean square error
print()
print('Final Root Mean Square Error on validation set: {}'.format(round(rmse_final, 3)))




# Oh no, we have run into our first problem...

AttributeError: module 'tensorflow._api.v2.train' has no attribute 'AdamOptimizer'

go to [Tensorflow-Keras](https://www.tensorflow.org/api_docs/python/tf) to see what the problem is with tf.train...

oh yeah it doesn't exist anymore (clearly this example was older)
It has since been update to be within tf.keras.optimizers

**Replace the tf.train.AdamOptimizer() in the build_model() function with an approriate Adam one found on the tensor flow site [TF](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)**

Rerun this model then the training script again, we will find another issue.

# Error 2

KeyError: 'val_mean_squared_error'
What does that mean

It is suggesting that the line, seen below, has an error

```
rmse_final = np.sqrt(float(hist['val_mean_squared_error'].tail(1)))
```

it is looking for that value inside hist that is called val_mean_squared_error

call the hist variable to see a print out of this variable


In [ ]:
hist


Not that there is no val_mean_squared_error only a val mse.
**Correct the above code and rerun.**

In [ ]:
rmse_final = np.sqrt(float(hist['val_mse'].tail(1)))
print()
print('Final Root Mean Square Error on validation set: {}'.format(round(rmse_final, 3)))

Now, let's plot the loss function measure on the training and validation sets. The validation set is used to prevent overfitting ([learn more about it here](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit)). However, because our network is small, the training convergence without noticeably overfitting the data as the plot shows.

In [ ]:
def plot_history():
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [Thousand Dollars$^2$]')
    plt.plot(hist['epoch'], hist['mean_squared_error'], label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_squared_error'], label = 'Val Error')
    plt.legend()
    plt.ylim([0,50])

plot_history()

Again we have an issue with the code and the naming of the mse terms,  replace the two above with mse

In [ ]:
# Define a function to plot the training and validation mean square error
def plot_history():
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [Thousand Dollars$^2$]')
    plt.plot(hist['epoch'], hist['mse'], label='Train Error')
    plt.plot(hist['epoch'], hist['val_mse'], label = 'Validation Error')
    plt.legend()
    plt.ylim([0,50])

# Call the function to plot the training and validation mean square error
plot_history()

*This code defines a function called "plot_history" which plots the training and validation mean square error as a function of the epochs. The x-axis is labeled as "Epoch" and y-axis is labeled as "Mean Square Error [Thousand Dollars$^2$]". The function plots the training error by using the data from the 'mse' column of the hist dataframe and plots the validation error by using the data from the 'val_mse' column of the hist dataframe. The plot is labeled with a legend, and y-axis limit is set to [0,50]. The function is called at the end of the code to execute the plot.*

Next, compare how the model performs on the test dataset:

In [ ]:
# Normalize the test features using the statistics from the training set
test_features_norm = (test_features - train_mean) / train_std

# Evaluate the model on the test set
mse, _, _ = model.evaluate(test_features_norm, test_labels)

# Calculate the root mean square error
rmse = np.sqrt(mse)

# Print the root mean square error
print('Root Mean Square Error on test set: {}'.format(round(rmse, 3)))

4/4 [==============================] - 0s 4ms/step - loss: 16.5975 - mae: 2.7258 - mse: 16.5975
Root Mean Square Error on test set: 4.074


Compare the RMSE measure you get to the [Kaggle leaderboard](https://www.kaggle.com/c/boston-housing/leaderboard). An RMSE of 4 puts us in 20th place.

## Improving the results
Change the number of neurons from 20 to a higher and lower value and note the changes.

Similairly change the optimiser from the tensorflow link used previous, to see if you can improve your result.

See if you can find how to add learning rate modifications to the model also available in the API doc link on the tensorflow page given earlier.










---





---





---

